# Data Processing and splitting

In [39]:
from logic.processing import load_data, preproc, balance_dataset, data_filter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
from sklearn.pipeline import make_pipeline
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import InputLayer,Dropout,BatchNormalization, Bidirectional,LSTM,Dense
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import pandas as pd

In [2]:
df_train = load_data('drugsComTrain_raw.csv')
df_test = load_data('drugsComTest_raw.csv')

In [3]:
df_train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [4]:
df_train_filter = data_filter(df_train)
df_test_filter = data_filter(df_test)

In [5]:
df_test_filter.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,sentiment,review_length
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22,1,68
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17,1,48
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35,1,143
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4,1,149
7,169852,Amitriptyline,Migraine Prevention,"""This has been great for me. I&#039;ve been on...",9,21-Apr-09,32,1,64


In [6]:
df_train_prep = preproc(df_train_filter)
df_test_prep = preproc(df_test_filter)

In [7]:
X_train = df_train_prep['clean']
y_train = df_train_prep['sentiment']

In [8]:
X_test = df_test_prep['clean']
y_test = df_test_prep['sentiment']

In [9]:
X_train_b,y_train_b = balance_dataset(X_train,y_train)
X_test_b,y_test_b = balance_dataset(X_test,y_test)

In [10]:
len(y_train_b[y_train_b==0])==len(y_train_b[y_train_b==1])

True

# Tokenize

In [11]:
X_train_tk = [text_to_word_sequence(_) for _ in X_train_b]

In [12]:
X_test_tk = [text_to_word_sequence(_) for _ in X_test_b]

# Embedding

In [13]:
word2vec = Word2Vec(sentences=X_train_tk, vector_size=60, min_count=10, window=10)

In [14]:
len(word2vec.wv.key_to_index)

6955

In [15]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [16]:
X_train_embed = embedding(word2vec, X_train_tk)
X_test_embed = embedding(word2vec, X_test_tk)

# Padding

In [17]:
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [18]:
X_train_pad.shape,X_test_pad.shape

((56314, 200, 60), (19092, 200, 60))

# Model Building

In [26]:
def init_model():
    model = Sequential()
    model.add(InputLayer((200,60)))
    model.add(layers.Masking(mask_value=0.))
    model.add(Bidirectional(LSTM(64, activation='tanh', return_sequences=False)))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [27]:
model = init_model()

In [28]:
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_2 (Masking)             │ (None, 200, 60)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        64,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,289 (266.75 KB)

 Trainable params: 68,225 (266.50 KB)

 Non-trainable params: 64 (256.00 B)

# Model training

In [30]:
es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train_b, 
          batch_size = 32,
          epochs=50,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 224s 171ms/step - accuracy: 0.7088 - loss: 0.5670 - val_accuracy: 0.8519 - val_loss: 0.3444
Epoch 2/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 191s 155ms/step - accuracy: 0.8567 - loss: 0.3333 - val_accuracy: 0.8700 - val_loss: 0.3158
Epoch 3/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 214s 165ms/step - accuracy: 0.8849 - loss: 0.2850 - val_accuracy: 0.8690 - val_loss: 0.3171
Epoch 4/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 235s 191ms/step - accuracy: 0.9036 - loss: 0.2456 - val_accuracy: 0.8952 - val_loss: 0.2636
Epoch 5/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 217s 176ms/step - accuracy: 0.9165 - loss: 0.2100 - val_accuracy: 0.8943 - val_loss: 0.2741
Epoch 6/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 203s 164ms/step - accuracy: 0.9335 - loss: 0.1758 - val_accuracy: 0.9039 - val_loss: 0.2543
Epoch 7/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 186s 151ms/step - accuracy: 0.9454 - loss: 0.1493 - val_accuracy: 0.9044 - val_loss: 0.2623
Epoch 8/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 196s 159ms/step - ac

# Evaluation

In [31]:
res = model.evaluate(X_test_pad, y_test_b, verbose=0)

In [32]:
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 90.305%


In [ ]:
y_pred = model.predict(X_test_pad)

In [ ]:
y_pred = np.round(y_pred)

In [ ]:
print(classification_report(y_test_b, y_pred))

# Demo test

In [49]:
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [50]:
def process(st):
        for punc in string.punctuation:
            st = st.replace(punc, '')
        ans = st.casefold().replace('\n', ' ')
        ansd = ''.join(x for x in ans if not x.isdigit())

        lemmaverb = [WordNetLemmatizer().lemmatize(word, pos='v') for word in ansd.split()]
        lemmanouns = [WordNetLemmatizer().lemmatize(word, pos='n') for word in lemmaverb]
        nans = ' '.join(lemmanouns)
        return nans

In [51]:
sentence = 'I had a terrible experience with this medication. After just a few days, I developed severe headaches and constant nausea. It didn’t improve my condition at all, and I had to stop taking it. I wouldn’t recommend it to anyone.'